In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os

In [6]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./Dados/annotations\Cars0.xml,226,419,125,173
1,./Dados/annotations\Cars1.xml,134,262,128,160
2,./Dados/annotations\Cars10.xml,140,303,5,148
3,./Dados/annotations\Cars100.xml,175,214,114,131
4,./Dados/annotations\Cars101.xml,167,240,202,220


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filepath  433 non-null    object
 1   xmin      433 non-null    int64 
 2   xmax      433 non-null    int64 
 3   ymin      433 non-null    int64 
 4   ymax      433 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 17.0+ KB


In [8]:
import xml.etree.ElementTree as xet

In [9]:
filename = df['filepath'][0]
filename

'./Dados/annotations\\Cars0.xml'

In [11]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./Dados/images', filename_image)
    
    return filepath_image

In [13]:
getFilename(filename)

'./Dados/images\\Cars0.png'

In [15]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./Dados/images\\Cars0.png',
 './Dados/images\\Cars1.png',
 './Dados/images\\Cars10.png',
 './Dados/images\\Cars100.png',
 './Dados/images\\Cars101.png',
 './Dados/images\\Cars102.png',
 './Dados/images\\Cars103.png',
 './Dados/images\\Cars104.png',
 './Dados/images\\Cars105.png',
 './Dados/images\\Cars106.png',
 './Dados/images\\Cars107.png',
 './Dados/images\\Cars108.png',
 './Dados/images\\Cars109.png',
 './Dados/images\\Cars11.png',
 './Dados/images\\Cars110.png',
 './Dados/images\\Cars111.png',
 './Dados/images\\Cars112.png',
 './Dados/images\\Cars113.png',
 './Dados/images\\Cars114.png',
 './Dados/images\\Cars115.png',
 './Dados/images\\Cars116.png',
 './Dados/images\\Cars117.png',
 './Dados/images\\Cars118.png',
 './Dados/images\\Cars119.png',
 './Dados/images\\Cars12.png',
 './Dados/images\\Cars120.png',
 './Dados/images\\Cars121.png',
 './Dados/images\\Cars122.png',
 './Dados/images\\Cars123.png',
 './Dados/images\\Cars124.png',
 './Dados/images\\Cars125.png',
 './Dados/image

### Verificando Image

In [16]:
file_path = image_path[0]
file_path

'./Dados/images\\Cars0.png'

In [19]:
img = cv2.imread(file_path)

cv2.namedWindow('exemplo', cv2.WINDOW_NORMAL)
cv2.imshow('exemplo', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
#xmin - ymin , xmax - ymax

cv2.rectangle(img, (226, 125), (419, 173), (0, 0, 255,), 3)
cv2.namedWindow('exemplo', cv2.WINDOW_NORMAL)
cv2.imshow('exemplo', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Processamento

In [21]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [22]:
labels = df.iloc[:,1:].values

In [32]:
data = []
output = []

for ind in range(len(image_path)):
    image = image_path[0]
    img_arr = cv2.imread(image)
    h, w, d = img_arr.shape

    load_image = load_img(image, target_size = (224, 224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr / 255.0

    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin / w, xmax / w
    nymin, nymax = ymin / h, ymax / h
    label_norm = (nxmin, nxmax, nymin, nymax)
    
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [40]:
X = np.array(data, dtype = np.float32)
y = np.array(output, dtype = np.float32)

In [41]:
X.shape, y.shape

((433, 224, 224, 3), (433, 4))

In [42]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

### Modelo Deep Learning

In [38]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [39]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 20s 0us/step


In [43]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

 conv2d_11 (Conv2D)          (None, 25, 25, 64)           12288     ['average_pooling2d[0][0]']   
                                                                                                  
 batch_normalization_5 (Bat  (None, 25, 25, 96)           288       ['conv2d_5[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_7 (Bat  (None, 25, 25, 64)           192       ['conv2d_7[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_10 (Ba  (None, 25, 25, 96)           288       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
          

 block35_1_conv (Conv2D)     (None, 25, 25, 320)          41280     ['block35_1_mixed[0][0]']     
                                                                                                  
 custom_scale_layer (Custom  (None, 25, 25, 320)          0         ['mixed_5b[0][0]',            
 ScaleLayer)                                                         'block35_1_conv[0][0]']      
                                                                                                  
 block35_1_ac (Activation)   (None, 25, 25, 320)          0         ['custom_scale_layer[0][0]']  
                                                                                                  
 conv2d_21 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_1_ac[0][0]']        
                                                                                                  
 batch_normalization_21 (Ba  (None, 25, 25, 32)           96        ['conv2d_21[0][0]']           
 tchNormal

                                                                                                  
 activation_25 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_25[0][0]
                                                                    ']                            
                                                                                                  
 activation_28 (Activation)  (None, 25, 25, 48)           0         ['batch_normalization_28[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_2_ac[0][0]']        
                                                                                                  
 conv2d_26 (Conv2D)          (None, 25, 25, 32)           9216      ['activation_25[0][0]']       
          

                                                                                                  
 activation_32 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_32[0][0]
                                                                    ']                            
                                                                                                  
 activation_35 (Activation)  (None, 25, 25, 64)           0         ['batch_normalization_35[0][0]
                                                                    ']                            
                                                                                                  
 block35_4_mixed (Concatena  (None, 25, 25, 128)          0         ['activation_30[0][0]',       
 te)                                                                 'activation_32[0][0]',       
                                                                     'activation_35[0][0]']       
          

...

 e)                                                                  'activation_170[0][0]']      
                                                                                                  
 block8_2_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_2_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_31 (Cus  (None, 5, 5, 2080)           0         ['block8_1_ac[0][0]',         
 tomScaleLayer)                                                      'block8_2_conv[0][0]']       
                                                                                                  
 block8_2_ac (Activation)    (None, 5, 5, 2080)           0         ['custom_scale_layer_31[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_17

 )                                                                  ]']                           
                                                                                                  
 block8_4_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_175[0][0]',      
 e)                                                                  'activation_178[0][0]']      
                                                                                                  
 block8_4_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_4_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_33 (Cus  (None, 5, 5, 2080)           0         ['block8_3_ac[0][0]',         
 tomScaleLayer)                                                      'block8_4_conv[0][0]']       
                                                                                                  
 block8_4_

 )                                                                  ]']                           
                                                                                                  
 activation_186 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_186[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_6_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_183[0][0]',      
 e)                                                                  'activation_186[0][0]']      
                                                                                                  
 block8_6_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_6_mixed[0][0]']      
                                                                                                  
 custom_sc

 atchNormalization)                                                                               
                                                                                                  
 activation_191 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_191[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_194 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_194[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_8_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_191[0][0]',      
 e)                                                                  'activation_194[0][0]']      
          

 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_202 (B  (None, 5, 5, 256)            768       ['conv2d_202[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_199 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_199[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_202 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_202[0][0
 )                                                                  ]']                           
          

### Treinamento

In [44]:
from tensorflow.keras.callbacks import TensorBoard

In [47]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test))

Epoch 1/100
35/35 [==============================] - 118s 3s/step - loss: 0.1380 - val_loss: 0.1349
Epoch 2/100
35/35 [==============================] - 103s 3s/step - loss: 0.1403 - val_loss: 0.1547
Epoch 3/100
35/35 [==============================] - 95s 3s/step - loss: 0.1429 - val_loss: 0.1348
Epoch 4/100
35/35 [==============================] - 98s 3s/step - loss: 0.1422 - val_loss: 0.1352
Epoch 5/100
35/35 [==============================] - 109s 3s/step - loss: 0.1400 - val_loss: 0.1375
Epoch 6/100
35/35 [==============================] - 106s 3s/step - loss: 0.1368 - val_loss: 0.1376
Epoch 7/100
35/35 [==============================] - 104s 3s/step - loss: 0.1363 - val_loss: 0.1349
Epoch 8/100
35/35 [==============================] - 104s 3s/step - loss: 0.1341 - val_loss: 0.1348
Epoch 9/100
35/35 [==============================] - 99s 3s/step - loss: 0.1376 - val_loss: 0.1487
Epoch 10/100
35/35 [==============================] - 99s 3s/step - loss: 0.1351 - val_loss: 0.1354
Epo

Epoch 83/100
35/35 [==============================] - 94s 3s/step - loss: 0.0342 - val_loss: 0.0407
Epoch 84/100
35/35 [==============================] - 94s 3s/step - loss: 0.0347 - val_loss: 0.0407
Epoch 85/100
35/35 [==============================] - 94s 3s/step - loss: 0.0345 - val_loss: 0.0406
Epoch 86/100
35/35 [==============================] - 95s 3s/step - loss: 0.0342 - val_loss: 0.0403
Epoch 87/100
35/35 [==============================] - 94s 3s/step - loss: 0.0341 - val_loss: 0.0404
Epoch 88/100
35/35 [==============================] - 95s 3s/step - loss: 0.0343 - val_loss: 0.0409
Epoch 89/100
35/35 [==============================] - 95s 3s/step - loss: 0.0338 - val_loss: 0.0413
Epoch 90/100
35/35 [==============================] - 110s 3s/step - loss: 0.0347 - val_loss: 0.0398
Epoch 91/100
35/35 [==============================] - 115s 3s/step - loss: 0.0340 - val_loss: 0.0398
Epoch 92/100
35/35 [==============================] - 96s 3s/step - loss: 0.0338 - val_loss: 0.039

In [49]:
model.save('./car_plate_detection.h5')

C:\Users\alfre\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.

